In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
Ddos_dataset = pd.read_csv(r"D:\target\ml\Cyber_attack\cybersecurity_attacks.csv")


In [3]:
Ddos_dataset.head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,...,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,...,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,NaN,Server
1,2020-08-26 07:08:30,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,...,Blocked,Low,Sumer Rana,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Bilaspur, Nagaland",NaN,Log Data,NaN,Firewall
2,2022-11-13 08:23:25,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,...,Ignored,Low,Himmat Karpe,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Segment C,"Bokaro, Rajasthan",114.133.48.179,Log Data,Alert Data,Firewall
3,2023-07-02 10:38:46,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,...,Blocked,Medium,Fateh Kibe,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_11_5; ...,Segment B,"Jaunpur, Rajasthan",NaN,NaN,Alert Data,Firewall
4,2023-07-16 13:11:07,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,...,Blocked,Low,Dhanush Chad,Mozilla/5.0 (compatible; MSIE 5.0; Windows NT ...,Segment C,"Anantapur, Tripura",149.6.110.119,NaN,Alert Data,Firewall


In [4]:
Ddos_dataset.shape

(40000, 25)

In [5]:
Ddos_dataset.describe()

,Source Port,Destination Port,Packet Length,Anomaly Scores
count,40000.000000,40000.000000,40000.000000,40000.000000
mean,32970.356450,33150.868650,781.452725,50.113473
std,18560.425604,18574.668842,416.044192,28.853598
min,1027.000000,1024.000000,64.000000,0.000000
25%,16850.750000,17094.750000,420.000000,25.150000
50%,32856.000000,33004.500000,782.000000,50.345000
75%,48928.250000,49287.000000,1143.000000,75.030000
max,65530.000000,65535.000000,1500.000000,100.000000


In [6]:
Ddos_dataset['Attack Type'].value_counts()

Attack Type
DDoS         13428
Malware      13307
Intrusion    13265
Name: count, dtype: int64

In [7]:
print(Ddos_dataset.columns)

Index(['Timestamp', 'Source IP Address', 'Destination IP Address',
       'Source Port', 'Destination Port', 'Protocol', 'Packet Length',
       'Packet Type', 'Traffic Type', 'Payload Data', 'Malware Indicators',
       'Anomaly Scores', 'Alerts/Warnings', 'Attack Type', 'Attack Signature',
       'Action Taken', 'Severity Level', 'User Information',
       'Device Information', 'Network Segment', 'Geo-location Data',
       'Proxy Information', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source'],
      dtype='object')


In [8]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd


le = LabelEncoder()
Ddos_dataset['Attack Type'] = le.fit_transform(Ddos_dataset['Attack Type'])

Ddos_dataset['Alerts/Warnings'] = Ddos_dataset['Alerts/Warnings'].apply(lambda x: 1 if x == 'Alert Triggered' else 0)

Ddos_dataset['Attack Signature'] = Ddos_dataset['Attack Type'].replace({'Known Pattern A': 0, 'Known Pattern B': 1})

Ddos_dataset['Traffic Type'] = Ddos_dataset['Traffic Type'].replace({'HTTP': 0, 'DNS': 1, 'FTP': 2})

Ddos_dataset['Packet Type'] = Ddos_dataset['Packet Type'].replace({'Data': 1, 'Control': 0})

Ddos_dataset['Protocol'] = Ddos_dataset['Protocol'].replace({'ICMP': 0, 'UDP': 1, 'TCP': 2})

Ddos_dataset['Severity Level'] = Ddos_dataset['Severity Level'].replace({'Low': 0, 'Medium': 1, 'High': 2})

Ddos_dataset['Malware Indicators'] = Ddos_dataset['Malware Indicators'].apply(lambda x: 1 if x == 'IoC Detected' else (0 if pd.isna(x) else x))


C:\Users\Asus\AppData\Local\Temp\ipykernel_25008\2271740946.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Ddos_dataset['Traffic Type'] = Ddos_dataset['Traffic Type'].replace({'HTTP': 0, 'DNS': 1, 'FTP': 2})
C:\Users\Asus\AppData\Local\Temp\ipykernel_25008\2271740946.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Ddos_dataset['Packet Type'] = Ddos_dataset['Packet Type'].replace({'Data': 1, 'Control': 0})
C:\Users\Asus\AppData\Local\Temp\ipykernel_25008\2271740946.py:16: FutureWarning: Downcasting behavior in `replace` 

In [9]:
X = Ddos_dataset.drop(columns=[
    'Timestamp', 'Source IP Address', 'Destination IP Address', 'Source Port',
    'Destination Port', 'Packet Length', 
     'Payload Data', 'Action Taken', 
    'User Information', 'Device Information', 'Network Segment',
    'Geo-location Data', 'Proxy Information', 'Firewall Logs',
    'IDS/IPS Alerts', 'Log Source','Attack Type'
])

Y = Ddos_dataset['Attack Type']

In [10]:
Ddos_dataset.describe()

,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Severity Level
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,32970.356450,33150.868650,0.996075,781.452725,0.494075,0.99760,0.500000,50.113473,0.498325,0.996975,0.996975,1.004975
std,18560.425604,18574.668842,0.817023,416.044192,0.499971,0.81585,0.500006,28.853598,0.500003,0.817547,0.817547,0.814934
min,1027.000000,1024.000000,0.000000,64.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16850.750000,17094.750000,0.000000,420.000000,0.000000,0.00000,0.000000,25.150000,0.000000,0.000000,0.000000,0.000000
50%,32856.000000,33004.500000,1.000000,782.000000,0.000000,1.00000,0.500000,50.345000,0.000000,1.000000,1.000000,1.000000
75%,48928.250000,49287.000000,2.000000,1143.000000,1.000000,2.00000,1.000000,75.030000,1.000000,2.000000,2.000000,2.000000
max,65530.000000,65535.000000,2.000000,1500.000000,1.000000,2.00000,1.000000,100.000000,1.000000,2.000000,2.000000,2.000000


In [11]:
X.head()

,Protocol,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Signature,Severity Level
0,0,1,0,1,28.67,0,2,0
1,0,1,0,1,51.50,0,2,0
2,1,0,0,1,87.42,1,0,0
3,1,1,0,0,15.79,1,2,1
4,2,1,1,0,0.52,1,0,0


In [12]:
Y.head()

0    2
1    2
2    0
3    2
4    0
Name: Attack Type, dtype: int32

In [13]:
print(X)
print(Y)

       Protocol  Packet Type  Traffic Type  Malware Indicators  \
0             0            1             0                   1   
1             0            1             0                   1   
2             1            0             0                   1   
3             1            1             0                   0   
4             2            1             1                   0   
...         ...          ...           ...                 ...   
39995         1            0             0                   1   
39996         1            0             0                   1   
39997         1            1             1                   1   
39998         1            1             2                   1   
39999         0            0             0                   0   

       Anomaly Scores  Alerts/Warnings  Attack Signature  Severity Level  
0               28.67                0                 2               0  
1               51.50                0                 2 

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
from sklearn.metrics import classification_report

# Assuming RF_MODEL is your trained Random Forest model and y_test, y_pred are defined
y_pred = rf_model.predict(X_test)  # Use the model to make predictions on the test set
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6653
           1       1.00      1.00      1.00      6713
           2       1.00      1.00      1.00      6634

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000



In [16]:

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

Accuracy: 1.0
Classification Report:
 <function classification_report at 0x000001E199CA96C0>


In [17]:

import joblib

# Assuming your model is named `model`
model_filename = 'cyber_attack_model.joblib'

# Save the model
joblib.dump(rf_model, model_filename)

['cyber_attack_model.joblib']

In [18]:
input_data=(	0,	1,	0,	1,	28.67,	0,	1,	0)
input_data_as_numpy_array = np.asarray(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
prediction = rf_model.predict(input_data_reshaped)
print(prediction)
if(prediction[0]==0):
    print('Ddos is detected')
elif(prediction[0]==1):
  print('Intrusion detected')
elif(prediction[0]==2):
  print('Malware detected')

[1]
Intrusion detected


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
